In [ ]:
bucket = 'xgboostbucket'
prefix = 'dataset'
key = 'dataset/credit_card_transactions-ibm_v2.csv'
# Define IAM role
import boto3
import re

import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder


import matplotlib.pyplot as plt
import os
import sagemaker
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput
from sagemaker.serializers import CSVSerializer
import io
s3 = boto3.client('s3')
role = get_execution_role()

In [ ]:
obj = s3.get_object(Bucket=bucket, Key=key)
df = pd.read_csv(obj.get("Body"),nrows=1000000)
df.head() 

Preprocessing:

In [ ]:
df['Is Fraud?']=df['Is Fraud?'].replace({'No':0,'Yes':1}) 
df['Errors?']=df['Errors?'].fillna('NAN')
df['Errors?']=df['Errors?'].apply(lambda value:value=='NAN')
df['Use Chip'].unique()
df['is online']=df['Use Chip'].apply(lambda value:value=='Online Transaction')
df['Use Chip']=df['Use Chip'].replace({'Swipe Transaction':0, 'Online Transaction':1, 'Chip Transaction':2})
df['Zip'] = df['Zip'].fillna(df['Zip'].mean())  
df['Amount'] = df['Amount'].apply(lambda value: float(value.split("$")[1]))
df['Hour'] = df['Time'].apply(lambda value: int(value.split(":")[0]))
df['Minutes'] = df['Time'].apply(lambda value: int(value.split(":")[1]))
df.drop(['Time'], axis=1, inplace=True)     
df['Merchant State']=df['Merchant State'].fillna('NAN')    
df['Merchant City']=df['Merchant City'].fillna('NAN') 
df['is vozmes']=df['Amount'].apply(lambda value: value<0)  
df['abs_amount']=df['Amount'].apply(lambda value: abs(value))
le=LabelEncoder() 
df['Merchant State']=le.fit_transform(df['Merchant State'])
le=LabelEncoder()
df['Merchant City']=le.fit_transform(df['Merchant City'])
df.drop('Merchant Name',axis=1,inplace=True)
df.drop('User',axis=1,inplace=True)
df = pd.concat([df['Is Fraud?'], df.drop(['Is Fraud?'], axis=1)], axis=1)
df.replace({False: 0, True: 1}, inplace=True)

In [ ]:
df.head(1)

In [ ]:
train_data, validation_data, test_data = np.split(df.sample(frac=1, random_state=42), [int(0.7 * len(df)), int(0.9 * len(df))])
train_data.to_csv('train.csv', header=False, index=False)
validation_data.to_csv('validation.csv', header=False, index=False)
test_data.to_csv('test.csv', header=False, index=False)

In [ ]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')
s3_input_train = TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = TrainingInput(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

Training:

In [ ]:
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'}

sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(containers[boto3.Session().region_name],
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(eta=0.1, objective='binary:logistic', num_round=25) 
xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

Deployment:

In [ ]:
xgb_predictor = xgb.deploy(
	initial_instance_count = 1,
	instance_type = 'ml.m4.xlarge',
	serializer = CSVSerializer())

Testing:

In [ ]:
def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])
        p = np.fromstring(predictions[1:], sep=',')
        p = np.round(p)

    return p

predictions = predict(test_data.to_numpy()[:,1:])
predictions

In [ ]:
import boto3
import numpy as np
# copy endpoint name u got from endpoint name print() or uploaded one and copy it here
ENDPOINT_NAME = 'xgboost-2022-05-10-20-38-39-998'
runtime = boto3.client('runtime.sagemaker')

def lambda_handler(event, context):
  inputs = event['data']
  result = []
  for input in inputs:
    serialized_input = ','.join(map(str, input))

    response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                     ContentType='text/csv',
                                     Body=serialized_input)
  
    result.append(response['Body'].read().decode())

  return result

input_json = { "data":
        [[0,2017,10,10,4.02,2,12095,146,29693.0,5814,1,0,11,19,0,4.02],
         [0,2017,10,10,4.02,2,12095,146,29693.0,5814,1,0,11,19,0,4.02],
         [0,2017,10,10,4.02,2,12095,146,29693.0,5814,1,0,11,19,0,4.02],
         [0,2017,10,10,4.02,2,12095,146,29693.0,5814,1,0,11,19,0,4.02]]
}

result = lambda_handler(input_json, _)
result